In [2]:
import json
import pandas as pd

from langchain_openai import ChatOpenAI
from aidial_rag_eval import evaluate
from aidial_rag_eval.metric_binds import CONTEXT_TO_ANSWER_INFERENCE,\
    ANSWER_TO_GROUND_TRUTH_INFERENCE, GROUND_TRUTH_TO_ANSWER_INFERENCE, ANSWER_REFUSAL, GROUND_TRUTH_REFUSAL

llm = ChatOpenAI(model="gpt-4o")

In [3]:
ground_truth_url = "./example_parquets/ground_truth.parquet"
answer_url = "./example_parquets/answer.parquet"
metrics_url = "./example_parquets/metrics.parquet"

In [4]:
evaluate(
    ground_truth_url,
    answer_url,
    metrics_url,
    llm=llm.with_retry(stop_after_attempt=4),
    metric_binds=[
        CONTEXT_TO_ANSWER_INFERENCE,
        ANSWER_TO_GROUND_TRUTH_INFERENCE,
        GROUND_TRUTH_TO_ANSWER_INFERENCE,
        ANSWER_REFUSAL,
        GROUND_TRUTH_REFUSAL,
    ],
    show_progress_bar=False,
)

Dataset(metadata_path='./example_parquets/metrics.parquet.metadata.json', data_path='./example_parquets/metrics.parquet', metadata=DatasetMetadata(created_at=datetime.datetime(2025, 4, 22, 10, 58, 25, 980271), sources=[Dataset(metadata_path='./example_parquets/ground_truth.parquet.metadata.json', data_path='./example_parquets/ground_truth.parquet', metadata=DatasetMetadata(created_at=None, sources=[], tools={}, file_info={'name': '/mnt/c/Users/Mikalai_Pechaneu/Work/dial-rag-eval/nli-branch-new/dial-rag-eval/examples/example_parquets/ground_truth.parquet', 'size': 4958, 'type': 'file', 'created': 1744772694.473407, 'islink': False, 'mode': 33279, 'uid': 1000, 'gid': 1000, 'mtime': 1744772694.473407, 'ino': 41095346599928296, 'nlink': 1}, metrics={}, statistics={})), Dataset(metadata_path='./example_parquets/answer.parquet.metadata.json', data_path='./example_parquets/answer.parquet', metadata=DatasetMetadata(created_at=None, sources=[], tools={}, file_info={'name': '/mnt/c/Users/Mikalai

In [5]:
df_metrics = pd.read_parquet(metrics_url)
df_metrics

,question,documents,facts,ground_truth_answer,context,answer,facts_ranks,context_relevance,context_highlight,recall,...,ans_gt_inference,ans_gt_json,ans_gt_highlight,gt_ans_inference,gt_ans_json,gt_ans_highlight,answer_refusal,ground_truth_refusal,mean_inference,median_inference
0,Who was the first president of the United States?,[george_washington],[George Washington was an American political l...,The first president of the United States was G...,[George Washington was an American political l...,George Washington was the first president of t...,[0],"[1, 0]","[{""match"": [{""text"": ""George Washington was an...",1.0,...,1.0,"[{""inference"": 1.0, ""hypothesis"": ""The first p...","{""corpus"": [{""text"": ""The first president of t...",1.0,"[{""inference"": 1.0, ""hypothesis"": ""George Wash...","{""corpus"": [{""text"": ""George Washington was th...",0.0,0.0,1.000000,1.0
1,When was Apple Inc. established and who was th...,[apple.pdf],"[Apple Inc. was established on April 1, 1976.,...",The establishment date of Apple Inc. is April ...,[Apple Inc. is an American multinational techn...,The CEO of Apple in 2020 was Tim Cook.,"[-1, 1]","[0, 1]","[{""match"": [{""text"": ""Apple Inc. is an America...",0.5,...,0.5,"[{""inference"": 0.5, ""hypothesis"": ""The establi...","{""corpus"": [{""text"": ""The establishment date o...",1.0,"[{""inference"": 1.0, ""hypothesis"": ""The CEO of ...","{""corpus"": [{""text"": ""The CEO of Apple in 2020...",0.0,0.0,0.833333,1.0
2,What is the capital city of Spain?,[spain],[Capital city of Spain is Madrid.],The capital city of Spain is Madrid.,[Barcelona is a city in Spain. It is the capit...,The capital city of Spain is Barcelona.,[-1],"[0, 0]","[{""match"": [{""text"": ""Barcelona is a city in S...",0.0,...,0.0,"[{""inference"": 0.0, ""hypothesis"": ""The capital...","{""corpus"": [{""text"": ""The capital city of Spai...",0.0,"[{""inference"": 0.0, ""hypothesis"": ""The capital...","{""corpus"": [{""text"": ""The capital city of Spai...",0.0,0.0,0.000000,0.0


In [6]:
df_metrics[["ctx_ans_inference", "ans_gt_inference", "gt_ans_inference"]]

,ctx_ans_inference,ans_gt_inference,gt_ans_inference
0,1.0,1.0,1.0
1,1.0,0.5,1.0
2,0.0,0.0,0.0


In [7]:
json.loads(df_metrics["ans_gt_json"][1])

[{'inference': 0.5,
  'hypothesis': 'The establishment date of Apple Inc. is April 1, 1976, and Tim Cook was the acting CEO in 2020.',
  'premise': ['When was Apple Inc. established and who was the CEO in 2020?\nThe CEO of Apple in 2020 was Tim Cook.'],
  'explanation': '[{"statement": "The establishment date of Apple Inc. is April 1, 1976.", "explanation": "The premise asks about the establishment date but does not provide the specific date.", "tag": "NEUT"}, {"statement": "Tim Cook was the acting CEO in 2020.", "explanation": "The premise states that Tim Cook was the CEO of Apple in 2020, which implies he was acting as CEO.", "tag": "ENT"}]'}]

In [8]:
import pandas as pd

In [9]:
df_gt = pd.read_parquet(ground_truth_url)
df_gt.head()

,question,documents,facts,answer
0,Who was the first president of the United States?,[george_washington],[George Washington was an American political l...,The first president of the United States was G...
1,When was Apple Inc. established and who was th...,[apple.pdf],"[Apple Inc. was established on April 1, 1976.,...",The establishment date of Apple Inc. is April ...
2,What is the capital city of Spain?,[spain],[Capital city of Spain is Madrid.],The capital city of Spain is Madrid.


In [10]:
df_a = pd.read_parquet(answer_url)
df_a.head()

,question,documents,context,answer
0,Who was the first president of the United States?,[george_washington],[George Washington was an American political l...,George Washington was the first president of t...
1,When was Apple Inc. established and who was th...,[apple.pdf],[Apple Inc. is an American multinational techn...,The CEO of Apple in 2020 was Tim Cook.
2,What is the capital city of Spain?,[spain],[Barcelona is a city in Spain. It is the capit...,The capital city of Spain is Barcelona.


In [11]:
from aidial_rag_eval import create_rag_eval_metrics_report

In [12]:
df_metrics = create_rag_eval_metrics_report(
    df_gt,
    df_a,
    llm=llm.with_retry(stop_after_attempt=4),
    metric_binds=[
        CONTEXT_TO_ANSWER_INFERENCE,
        ANSWER_TO_GROUND_TRUTH_INFERENCE,
        GROUND_TRUTH_TO_ANSWER_INFERENCE,
    ],
    show_progress_bar=False,
)

In [13]:
df_metrics.head()

,question,documents,facts,ground_truth_answer,context,answer,facts_ranks,context_relevance,context_highlight,recall,...,ctx_ans_json,ctx_ans_highlight,ans_gt_inference,ans_gt_json,ans_gt_highlight,gt_ans_inference,gt_ans_json,gt_ans_highlight,mean_inference,median_inference
0,Who was the first president of the United States?,[george_washington],[George Washington was an American political l...,The first president of the United States was G...,[George Washington was an American political l...,George Washington was the first president of t...,[0],"[1, 0]","[{""match"": [{""text"": ""George Washington was an...",1.0,...,"[{""inference"": 1.0, ""hypothesis"": ""George Wash...","{""corpus"": [{""text"": ""George Washington was th...",1.0,"[{""inference"": 1.0, ""hypothesis"": ""The first p...","{""corpus"": [{""text"": ""The first president of t...",1.0,"[{""inference"": 1.0, ""hypothesis"": ""George Wash...","{""corpus"": [{""text"": ""George Washington was th...",1.000000,1.0
1,When was Apple Inc. established and who was th...,[apple.pdf],"[Apple Inc. was established on April 1, 1976.,...",The establishment date of Apple Inc. is April ...,[Apple Inc. is an American multinational techn...,The CEO of Apple in 2020 was Tim Cook.,"[-1, 1]","[0, 1]","[{""match"": [{""text"": ""Apple Inc. is an America...",0.5,...,"[{""inference"": 1.0, ""hypothesis"": ""The CEO of ...","{""corpus"": [{""text"": ""The CEO of Apple in 2020...",0.5,"[{""inference"": 0.5, ""hypothesis"": ""The establi...","{""corpus"": [{""text"": ""The establishment date o...",1.0,"[{""inference"": 1.0, ""hypothesis"": ""The CEO of ...","{""corpus"": [{""text"": ""The CEO of Apple in 2020...",0.833333,1.0
2,What is the capital city of Spain?,[spain],[Capital city of Spain is Madrid.],The capital city of Spain is Madrid.,[Barcelona is a city in Spain. It is the capit...,The capital city of Spain is Barcelona.,[-1],"[0, 0]","[{""match"": [{""text"": ""Barcelona is a city in S...",0.0,...,"[{""inference"": 0.0, ""hypothesis"": ""The capital...","{""corpus"": [{""text"": ""The capital city of Spai...",0.0,"[{""inference"": 0.0, ""hypothesis"": ""The capital...","{""corpus"": [{""text"": ""The capital city of Spai...",0.0,"[{""inference"": 0.0, ""hypothesis"": ""The capital...","{""corpus"": [{""text"": ""The capital city of Spai...",0.000000,0.0
